# Notes: 

This notebook is a bit overly-complicated but the bulk of it all just has to do with data organization. Since this project is winding down I'm not going to clean this up to shorten the code substantially but rather let this here for posterity. All these metrics *should* have been pre-computed so if you have access to the results data it will be unnecessary to run the code in this notebook unless we have new models what we want to assess.

# Basic imports

In [2]:
import glob
import numpy as np
import json
from collections import defaultdict
import pandas as pd

# Scoring structure models with `QMEANDisCo`

**This code submits jobs and accesses QMEANDisCO software using the programatic API rather than downloading and running it on my own computer. At a certain point, it probably would be best to download/install the software.**

The server is located here: https://swissmodel.expasy.org/qmean/

And a link to the paper is here: https://doi.org/10.1093/bioinformatics/btz828

In my experience, a submission to this server takes ~5 minutes to complete so I space out individual requests so as not to overload their servers or get flagged. In order to submit too many jobs or to perform this step too frequently, the first thing that I do is check whether the relevant files already exist on my computer (assuming a very particular directory structure) and if so I skip them and only submit jobs for new proteins.

In [4]:
#Libraries to handle internet requests and to manage the file system
import requests
import time
import os

qmean_url = 'https://swissmodel.expasy.org/qmean/submit/'

## First working through the (initial) IF2 and EFTU structures from `trRosetta`

**Basically, I first cycle through and send a lot of requests to the `qmeandisco` server. I compile all the results into a dictionary.**

In [ ]:
###This is where I want to put results
save_dir = '../Results/QMEAN/trRosetta/'
assert os.path.exists(save_dir)

###Send requests (if I don't already have them)
response_dict = {}
for base_dir in glob.glob('../Results/trRosetta/EFTU_*') + glob.glob('../Results/trRosetta/IF2_*'):

    ##############################################################################################
    ###Note: need to manually cycle through model1, model2, ... model 5 below to finish all models
    my_file_loc = base_dir + '/model1.pdb'
    ##############################################################################################
    
    ###This is where I'm going to save the new QMEANDisCo file
    f_out = my_file_loc.split('/')[-2] + '_' + my_file_loc.split('/')[-1]
    f_out = f_out.split('.')[0]
    print(base_dir, f_out)
    ###This code says to start the loop over if this file already exists
    if os.path.exists('{}{}.json'.format(save_dir, f_out)):
        print('Already finished this one')
        continue
    
    ###Send the actual request
    response = requests.post(url=qmean_url,
                         data={'email': 'adam.hockenberry@utexas.edu'},
                         files={'structure': open(my_file_loc,'r')})
    ###Store the response for now
    response_dict[my_file_loc] = response
    ###And take a little nap before moving on
    time.sleep(360)

**Now to process / download the results from that dictionary from all the initial queries that have (hopefully) completed their processing**

In [ ]:
###Iterate through items in the dictionary
for key, value in response_dict.items():
    print(key, value.status_code)
    
    ###Determine where I'm saving the file to (again)
    f_out = key.split('/')[-2] + '_' + key.split('/')[-1]
    f_out = f_out.split('.')[0]
    print(f_out)
    
    ###Actually save the '.json' file
    try:
        current_status = requests.get(value.json()["results_json"])
        results = current_status.json()
        with open('{}{}.json'.format(save_dir, f_out), 'w') as outfile:
            json.dump(results, outfile)
        ###Take a shorter nap
        time.sleep(60)
    except:
        print('Did not work, manually fix/investigate')

## Now going through the (initial) IF2 and EFTU structures from `I-TASSER`

This code is basically identical to the above few code cells so I'm not annotating as much here

In [ ]:
save_dir = '../Results/QMEAN/ITASSER/'
assert os.path.exists(save_dir)

response_dict = {}
for base_dir in glob.glob('../Results/ITASSER/IF2_*')+glob.glob('../Results/ITASSER/EFTU_*')[:]:
    ###Remember to manually change the model numbers to run on both model1 and model2
    
    ##############################################################################################
    my_file_loc = base_dir + '/model2.pdb' 
    ##############################################################################################


    f_out = my_file_loc.split('/')[-2] + '_' + my_file_loc.split('/')[-1]
    f_out = f_out.split('.')[0]
    print(base_dir, f_out)
    if os.path.exists('{}{}.json'.format(save_dir, f_out)):
        print('Already finished this one')
        continue
    response = requests.post(url=qmean_url,
                         data={'email': 'adam.hockenberry@utexas.edu'},
                         files={'structure': open(my_file_loc,'r')})
    response_dict[my_file_loc] = response
    time.sleep(240)

In [ ]:
###Process request outputs and save
for key, value in response_dict.items():
    print(key, value.status_code)
    f_out = key.split('/')[-2] + '_' + key.split('/')[-1]
    f_out = f_out.split('.')[0]
    print(f_out)
    try:
        current_status = requests.get(value.json()["results_json"])
        results = current_status.json()
        with open('{}{}.json'.format(save_dir, f_out), 'w') as outfile:
            json.dump(results, outfile)
        time.sleep(60)
    except:
        print('Did not work, manually fix/investigate')

## Now running analyses on the "clean" template files from `PDB`

Again the code is pretty much identical to the above sections so not annotating much here

In [ ]:
save_dir = '../Results/QMEAN/empirical/'
assert os.path.exists(save_dir)

response_dict = {}
for my_file_loc in glob.glob('../Data/Structures/Template_structures/*.clean.pdb')[:]:
    f_out = 'Templates' + '_' + my_file_loc.split('/')[-1]
    f_out = f_out.split('.')[0]
    print(f_out)
    print(base_dir, f_out)
    if os.path.exists('{}{}.json'.format(save_dir, f_out)):
        print('Already finished this one')
        continue
    response = requests.post(url=qmean_url,
                         data={'email': 'adam.hockenberry@utexas.edu'},
                         files={'structure': open(my_file_loc,'r')})
    response_dict[my_file_loc] = response
    time.sleep(240)

In [ ]:
###Process request outputs and save
for key, value in response_dict.items():
    print(key, value.status_code)
    f_out = key.split('/')[-2] + '_' + key.split('/')[-1]
    f_out = f_out.split('.')[0]
    print(f_out)
    try:
        current_status = requests.get(value.json()["results_json"])
        results = current_status.json()
        with open('{}{}.json'.format(save_dir, f_out), 'w') as outfile:
            json.dump(results, outfile)
        time.sleep(60)
    except:
        print('Did not work, manually fix/investigate')

## Finally, the `modeller` models

**To keep things consistent with some of the programs I'm going to re-name the files here as well when I write them so that I have model1.pdb, model2.pdb, *etc.***

In [ ]:
from io import StringIO
import shutil

In [ ]:
res_dir = '../Results/modeller/'
for base_dir in glob.glob('../Data/Structures/Anc_Structures_v2/IF2_*')[:]:
    if 'IF2_Anc559' in base_dir:
        continue
    if '1zo1' in base_dir:
        continue
    try:
        log_file = base_dir + '/model-single.log'
        with open(log_file, 'r') as infile:
            tempy = infile.readlines()[-6:]
    except FileNotFoundError:
        log_file = base_dir + '/model_mult.log'
        with open(log_file, 'r') as infile:
            tempy = infile.readlines()[-6:]
    df = pd.read_csv(StringIO(''.join(tempy)), delim_whitespace=True, header=None)
    if 'model-single' in log_file:
        df = df.sort_values(2)
    elif 'model_mult' in log_file:
        df = df.sort_values(1)
    else:
        print('MAJOR ERROR HERE')
        break
    assert df.shape[0] == 5
    for i, index in enumerate(list(df.index)):
        old_file_loc = base_dir + '/' + df.loc[index][0]
        new_file_loc = base_dir + '/' + 'model{}.pdb'.format(i+1)
        print(old_file_loc, new_file_loc)
        ##Make new copies within the original Data folder
        shutil.copy(old_file_loc, new_file_loc)
        ##And copy over to the Results folder for consistency
        newer_file_loc = new_file_loc.replace('/Data/Structures/Anc_Structures_v2/', '/Results/modeller/')
        if not os.path.exists(os.path.dirname(newer_file_loc)):
            os.makedirs(os.path.dirname(newer_file_loc), exist_ok=True)
        shutil.copy(new_file_loc, newer_file_loc)

**Now go through that same old process of sending away queries and holding the results**

In [ ]:
save_dir = '../Results/QMEAN/modeller/'
assert os.path.exists(save_dir)

response_dict = {}
for base_dir in glob.glob('../Results/modeller/*')[:]:
    
    ##############################################################################################
    my_file_loc = base_dir + '/model5.pdb' ###Manually specify model numbers
    ##############################################################################################

    ###If you know there are particular files you want to ignore you can use this type of an approach
#     if '/IF2_Anc702_3jcj_1efc/' not in my_file_loc and '/IF2_Anc719_3jcj_1efc/' not in my_file_loc:
#         continue
    
    f_out = my_file_loc.split('/')[-2] + '_' + my_file_loc.split('/')[-1]
    f_out = f_out.split('.')[0]
    print(base_dir, f_out)
    if os.path.exists('{}{}.json'.format(save_dir, f_out)):
        print('Already finished this one')
        continue
    response = requests.post(url=qmean_url,
                         data={'email': 'adam.hockenberry@utexas.edu'},
                         files={'structure': open(my_file_loc,'r')})
    response_dict[my_file_loc] = response
    time.sleep(360)

**And finally, downloading them into their proper location**

In [ ]:
###Process request outputs and save
for key, value in response_dict.items():
    print(key, value.status_code)
    f_out = key.split('/')[-2] + '_' + key.split('/')[-1]
    f_out = f_out.split('.')[0]
    print(f_out)
    try:
        current_status = requests.get(value.json()["results_json"])
        results = current_status.json()
        with open('{}{}.json'.format(save_dir, f_out), 'w') as outfile:
            json.dump(results, outfile)
        time.sleep(60)
    except:
        print('Did not work, manually fix/investigate')

# Assemble structure quality data from *all* sources into a dataframe/s

In [ ]:
base_molecule_list = []
molecule_id_list = []
model_number_list = []
template_id_list = []
qmean_score_list = []
source_list = []

for results_dir in glob.glob('../Results/QMEAN/*'):
    print(results_dir)
    source = results_dir.split('/')[-1]
    method = results_dir.split('/')[-1]
    for json_loc in glob.glob(results_dir + '/*.json'):
        print(json_loc)
        structure_name = json_loc.split('/')[-1]
        if source == 'empirical':
            molecule_id = structure_name.split('_')[1].split('.json')[0]
            if molecule_id in ['1zo1', '3jcjF']:
                base_molecule = 'IF2'
            elif molecule_id in ['1efc', '6gfu']:
                base_molecule = 'EFTU'
            else:
                raise Exception('Major error in if statement')
            model_number = ''
            template_id = ''
        else:
            base_molecule = structure_name.split('_')[0]
            molecule_id = structure_name.split('_')[1]
            if method == 'modeller':
                if len(structure_name.split('_')) == 3:
                    if base_molecule == 'IF2':
                        template_id = '1zo1'
                    elif base_molecule == 'EFTU':
                        template_id = '1efc'
                    else:
                        raise Exception('Major error in if statement number 2')
                    model_number = structure_name.split('_')[2].split('.')[0]
                    
                elif len(structure_name.split('_')) == 4:
                    template_id = structure_name.split('_')[2]
                    model_number = structure_name.split('_')[3].split('.')[0]
                    
                elif len(structure_name.split('_')) == 5:
                    template_id = '_'.join([structure_name.split('_')[2], structure_name.split('_')[3]])
                    model_number = structure_name.split('_')[4].split('.')[0]
                else:
                    raise Exception('Major error in if statement number 3')
            else:
                template_id = ''
                model_number = structure_name.split('_')[2].split('.')[0]

            print('####', base_molecule, molecule_id, model_number, template_id)
            
        with open(json_loc, 'r') as infile:
            results = json.load(infile)
        my_metric = results['models']['model_001']['scores']['global_scores']['avg_local_score']

        base_molecule_list.append(base_molecule)
        molecule_id_list.append(molecule_id)
        template_id_list.append(template_id)
        model_number_list.append(model_number)
        source_list.append(source)
        qmean_score_list.append(my_metric)

In [ ]:
df = pd.DataFrame(zip(base_molecule_list, molecule_id_list, template_id_list, model_number_list, source_list, qmean_score_list))
df.columns = ['protein', 'variant', 'template', 'model_number', 'source', 'qmeandisco_score']
df = df.sort_values(['source', 'protein', 'variant', 'model_number', 'qmeandisco_score'])
df = df.reset_index(drop=True)
print(df.shape)
df.head(n=20)

In [ ]:
df[df['source']=='modeller']['template'].value_counts()

In [ ]:
df.to_csv('../Results/QMEAN/full_results.tsv', sep='\t')

**Select only the top `QMEANDisCo` model and save as a separate `.tsv` file**

In [ ]:
best_df = df.loc[df.groupby(['protein', 'variant', 'source'], sort=False)['qmeandisco_score'].idxmax()] 
best_df = best_df.sort_values(['protein', 'variant', 'model_number', 'source', 'qmeandisco_score'])
best_df.reset_index(drop=True)

In [ ]:
best_df.to_csv('../Results/QMEAN/best_results.tsv', sep='\t')

## First, comparing different modeller templates

In [ ]:
###IF2 plots
protein = 'IF2'
model_labels_1 = ['Anc520',
               'Anc536',
               'Anc550',
               'Anc558',
               'Anc622',
               'Anc670',
               'Anc702',
               'Anc719',
               'Anc743',
               'Anc750',
               'Anc754',
               'Anc788',
                'Anc789']
model_labels_3 = ['1zo1', '3jcjF']

all_model_labels = model_labels_1 + model_labels_3
x_vals = list(range(len(all_model_labels)))

In [ ]:
####Best plot
fig, ax = plt.subplots(figsize=(5,2))
x_offset=-0.1
method = 'modeller'
templates = ['1zo1', '3jcj', '3jcj_1efc']
for i, template in enumerate(templates):
    print(template)
    temp_df = df[(df['source']==method)&(df['protein']==protein)]
    temp_df = temp_df[temp_df['template']==template]
    print(temp_df.shape)
    y_vals_1 = []
    y_vals_2 = []
    for label in model_labels_1:
        proposed_df = temp_df[temp_df['variant']==label]
        proposed_df = proposed_df.sort_values('qmeandisco_score', ascending=False)
        if proposed_df.shape[0] >= 1:
            y_vals_1.append(proposed_df.iloc[0]['qmeandisco_score'])
        else:
            y_vals_1.append(np.nan)
    
    ax.plot(np.array(x_vals[:len(model_labels_1)])+x_offset,
            y_vals_1,
            label='Template={}'.format(template),
            color=colors[i],
            marker='s')
    x_offset += 0.1


temp_df = best_df[(best_df['source']=='empirical')&(best_df['protein']==protein)]
y_vals_3 = []
for label in model_labels_3:
    proposed_df = temp_df[temp_df['variant']==label]
    if proposed_df.shape[0] == 1:
        y_vals_3.append(proposed_df.iloc[0]['qmeandisco_score']) 
    else:
        y_vals_3.append(np.nan)
ax.plot(x_vals[len(model_labels_1):],
        y_vals_3,
        color=colors[i+1],
        linestyle='',
        marker='s',
        label='Empirical')
    
ax.set_ylim(0, 1)
ax.axvline(len(x_vals)-len(model_labels_3)-0.5, c='k')
ax.set_xticks(x_vals)
ax.set_xticklabels(all_model_labels, rotation=90);
ax.set_ylabel('QMEANDisCo (max)')
# ax.legend(ncol=2, loc='best')
handles, labels = plt.gca().get_legend_handles_labels()
order = [0, 1, 2, 3]
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], ncol=2)

## Next, plotting only the best model for all cases with multiple models

In [ ]:
###IF2 plots
protein = 'IF2'
model_labels_1 = ['Anc520',
               'Anc536',
               'Anc550',
               'Anc558',
               'Anc622',
               'Anc670',
               'Anc702',
               'Anc719',
               'Anc743',
               'Anc750',
               'Anc754',
               'Anc788']
model_labels_2 = ['1zo1', '3jcjF']
model_labels_3 = ['1zo1', '3jcjF']


# ##EFTU plots
# protein = 'EFTU'
# model_labels_1 = ['Anc168',
#                'Anc262']
# model_labels_2 = ['1efc']
# model_labels_3 = ['1efc', '6gfu']


all_model_labels = model_labels_1 + model_labels_2 + model_labels_3
x_vals = list(range(len(all_model_labels)))

In [ ]:
####Best plot
fig, ax = plt.subplots(figsize=(5,2))
x_offset=-0.1
methods = ['trRosetta', 'ITASSER', 'modeller']
for i, method in enumerate(methods):
    temp_df = best_df[(best_df['source']==method)&(best_df['protein']==protein)]
    y_vals_1 = []
    y_vals_2 = []
    for label in model_labels_1:
        proposed_df = temp_df[temp_df['variant']==label]
        if proposed_df.shape[0] == 1:
            y_vals_1.append(proposed_df.iloc[0]['qmeandisco_score'])
        else:
            y_vals_1.append(np.nan)
    for label in model_labels_2:
        proposed_df = temp_df[temp_df['variant']==label]
        if proposed_df.shape[0] == 1:
            y_vals_2.append(proposed_df.iloc[0]['qmeandisco_score']) 
        else:
            y_vals_2.append(np.nan)
    
    ax.plot(np.array(x_vals[:len(model_labels_1)])+x_offset,
            y_vals_1,
            label=method,
            color=colors[i],
            marker='s')
    
    ax.plot(np.array(x_vals[len(model_labels_1):len(model_labels_1)+len(model_labels_2)])+x_offset,
            y_vals_2,
            color=colors[i],
            linestyle='',
            marker='s')
    x_offset += 0.1


temp_df = best_df[(best_df['source']=='empirical')&(best_df['protein']==protein)]
y_vals_3 = []
for label in model_labels_3:
    proposed_df = temp_df[temp_df['variant']==label]
    if proposed_df.shape[0] == 1:
        y_vals_3.append(proposed_df.iloc[0]['qmeandisco_score']) 
    else:
        y_vals_3.append(np.nan)
ax.plot(x_vals[len(model_labels_1)+len(model_labels_2):],
        y_vals_3,
        color=colors[i+1],
        linestyle='',
        marker='s',
        label='Empirical')
    
ax.set_ylim(0, 1)
ax.axvline(len(x_vals)-len(model_labels_3)-0.5, c='k')
ax.axvline(len(x_vals)-len(model_labels_2)-len(model_labels_3)-0.5, c='k')
ax.set_xticks(x_vals)
ax.set_xticklabels(all_model_labels, rotation=90);
ax.set_ylabel('QMEANDisCo (max)')
# ax.legend(ncol=2, loc='best')
handles, labels = plt.gca().get_legend_handles_labels()
order = [0, 1, 2, 3]
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], ncol=2)

## Now, plotting means plus standard deviations (when possible)

In [ ]:
####Best plot
fig, ax = plt.subplots(figsize=(5,2))

methods = ['trRosetta', 'ITASSER', 'modeller']
x_offset=-0.1
for i, method in enumerate(methods):
    temp_df = df[(df['source']==method)&(df['protein']==protein)]
    y_vals_1 = []
    y_errs_1 = []
    y_vals_2 = []
    y_errs_2 = []
    for label in model_labels_1:
        proposed_df = temp_df[temp_df['variant']==label]
        if proposed_df.shape[0] > 0:
            y_vals_1.append(proposed_df['qmeandisco_score'].mean())
            y_errs_1.append(proposed_df['qmeandisco_score'].std())
        else:
            y_vals_1.append(np.nan)
            y_errs_1.append(np.nan)
    for label in model_labels_2:
        proposed_df = temp_df[temp_df['variant']==label]
        if proposed_df.shape[0] > 0:
            y_vals_2.append(proposed_df['qmeandisco_score'].mean())
            y_errs_2.append(proposed_df['qmeandisco_score'].std())
        else:
            y_vals_2.append(np.nan)
            y_errs_2.append(np.nan)
    
    ax.errorbar(np.array(x_vals[:len(model_labels_1)])+x_offset,
            y_vals_1,
            yerr=y_errs_1,
            label=method,
            color=colors[i],
            marker='s')
    ax.errorbar(np.array(x_vals[len(model_labels_1):len(model_labels_1)+len(model_labels_2)])+x_offset,
            y_vals_2,
            yerr=y_errs_2,
            color=colors[i],
            linestyle='',
            marker='s')
    x_offset += 0.1
temp_df = best_df[(best_df['source']=='empirical')&(best_df['protein']==protein)]
y_vals_3 = []
for label in model_labels_3:
    proposed_df = temp_df[temp_df['variant']==label]
    if proposed_df.shape[0] == 1:
        y_vals_3.append(proposed_df.iloc[0]['qmeandisco_score']) 
    else:
        y_vals_3.append(np.nan)
ax.plot(x_vals[len(model_labels_1)+len(model_labels_2):],
        y_vals_3,
        color=colors[i+1],
        linestyle='',
        marker='s',
        label='Empirical')
    
ax.set_ylim(0, 1)
ax.axvline(len(x_vals)-len(model_labels_3)-0.5, c='k')
ax.axvline(len(x_vals)-len(model_labels_2)-len(model_labels_3)-0.5, c='k')
ax.set_xticks(x_vals)
ax.set_xticklabels(all_model_labels, rotation=90);
ax.set_ylabel('QMEANDisCo (mean)')

handles, labels = plt.gca().get_legend_handles_labels()
order = [1, 2, 3, 0]
ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order], ncol=2)

# Scratch